# Clasa Game

In [2]:
import numpy as np

class Game:
    MAX = 'x'
    MIN = '0'
    GOL = '.'
    
    MORI_POSIBILE = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8), (9, 10, 11), (12, 13, 14),
        (15, 16, 17), (18, 19, 20), (21, 22, 23),
        (0, 9, 21), (3, 10, 18), (6, 11, 15), (1, 4, 7), (16, 19, 22),
        (8, 12, 17), (5, 13, 20), (2, 14, 23)
    ]

    @classmethod
    def jucator_opus(self, jucator):
        return Game.MIN if jucator == Game.MAX else Game.MAX

# Clasa Nod

In [3]:
class Nod:
    def __init__(self, informatie=np.array([Game.GOL] * 24), parinte=None, jucator=Game.MAX, piese_de_pus=(0, 0)):
        self.informatie = informatie
        self.parinte = parinte
        self.jucator = jucator
        self.piese_de_pus = piese_de_pus
        self.euristica = 0

    def __repr__(self):
        return str(self.informatie)

    def __str__(self):
        
        b = self.informatie
        
        board = (
            f"    {b[0]}-----------{b[1]}-----------{b[2]}\n"
            f"    |           |           |\n"
            f"    |   {b[3]}-------{b[4]}-------{b[5]}   |\n"
            f"    |   |       |       |   |\n"
            f"    |   |   {b[6]}---{b[7]}---{b[8]}   |   |\n"
            f"    |   |   |       |   |   |\n"
            f"    {b[9]}---{b[10]}---{b[11]}       {b[12]}---{b[13]}---{b[14]}\n"
            f"    |   |   |       |   |   |\n"
            f"    |   |   {b[15]}---{b[16]}---{b[17]}   |   |\n"
            f"    |   |       |       |   |\n"
            f"    |   {b[18]}-------{b[19]}-------{b[20]}   |\n"
            f"    |           |           |\n"
            f"    {b[21]}-----------{b[22]}-----------{b[23]}\n"
            f"\nPiese de pus: x={self.piese_de_pus[0]}, 0={self.piese_de_pus[1]}\n"
            #f"\nRandul jucatorului: {self.jucator}\n"
        )
        return board

    def __eq__(self, other):
        return np.array_equal(self.informatie, other.informatie)

    def __lt__(self, other):
        if other is None:
            return True
        return self.euristica <= other.euristica
    
    def drumRadacina(self):
        ''' Calculeaza lista nodurilor de la radacina pana la nodul curent. '''
        if self.parinte is None:
            return np.array(self)
        return np.append(self.parinte.drumRadacina(), self)
    
    def succesori(self):
        ''' Calculeaza lista succesorilor directi ai starii curente.
        :return: lista starilor admisibile
        '''
        succesori = []
        n_x, n_0 = self.piese_de_pus
        next_piese = (n_x - 1, n_0) if self.jucator == Game.MAX else (n_x, n_0 - 1)

        for i in range(24):
            if self.informatie[i] == Game.GOL:
                info_noua = self.informatie.copy()
                info_noua[i] = self.jucator
                succesor = Nod(info_noua, self, Game.jucator_opus(self.jucator), next_piese)
                succesori.append(succesor)
        return succesori

    # Verifica daca jucatorul "simbol" a facut o moara.
    def check_moara(self, simbol):
        for moara in Game.MORI_POSIBILE:
            if all(self.informatie[pos] == simbol for pos in moara):
                return True
        return False

    # Verifica daca nodul curent este nod scop, adica daca este o stare finala a jocului
    def scop(self):
        if self.check_moara(Game.MAX):
            return Game.MAX
        if self.check_moara(Game.MIN):
            return Game.MIN
        if self.piese_de_pus[0] == 0 and self.piese_de_pus[1] == 0 and Game.GOL not in self.informatie:
            return 'remiza'
        return None

    # Calculeaza scorul unei secvente raprotat la jucatorul curent
    # scor secventa = 10 ^ (nr simboluri in secventa - 1) daca jucatorul opus nu e in secventa, 0 altfel
    def scor_secventa(self, secventa, simbol):
        if Game.jucator_opus(simbol) in secventa:
            return 0
        count = sum(1 for s in secventa if s == simbol)
        return 10 ** (count - 1) if count > 0 else 0

    # Scorul total al unei stari
    def estimare(self):
        punctaj = 0
        for moara in Game.MORI_POSIBILE:
            secventa = [self.informatie[pos] for pos in moara]
            punctaj += self.scor_secventa(secventa, Game.MAX)
            punctaj -= self.scor_secventa(secventa, Game.MIN)
        self.euristica = punctaj
        return punctaj

# Clasa Graf

In [4]:
class Graf:
    def __init__(self, stare_initiala, piese_de_pus, jucator_curent):
        self.nod_initial = Nod(np.array(list(stare_initiala)), None, jucator_curent, piese_de_pus)
        

    def succesori(self, nod):
        ''' Primeste un nod al arborelui de parcurgere si returnează lista
        succesorilor directi ai nodului care nu au fost vizitati pe ramura curenta.

        :param nod: un nod oarecare
        :return: lista succesorilor valizi ai nodului
        '''
        succesori = nod.succesori()
        for nodCurent in succesori:
            nodCurent.parinte = nod
        
        return succesori

    def minmax(self, nod, adancime, joacaMax):
        if adancime == 0 or nod.scop():
            nod.estimare()
            return nod

        lista_succesori = self.succesori(nod)
        if nod.jucator == Game.MAX:
            best_nod = max([self.minmax(succesor, adancime - 1, not joacaMax) for succesor in lista_succesori])
        else:
            best_nod = min([self.minmax(succesor, adancime - 1, not joacaMax) for succesor in lista_succesori])
        
        return best_nod

    def alphabeta(self, nod, adancime, alpha, beta):
        if adancime == 0 or nod.scop():
            return nod.estimare(), nod

        if nod.jucator == Game.MAX:
            max_eval = -float('inf')
            best_nod = None
            for fiu in nod.succesori():
                eval, _ = self.alphabeta(fiu, adancime - 1, alpha, beta)
                if eval > max_eval:
                    max_eval = eval
                    best_nod = fiu
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return max_eval, best_nod
        else:
            min_eval = float('inf')
            best_nod = None
            for fiu in nod.succesori():
                eval, _ = self.alphabeta(fiu, adancime - 1, alpha, beta)
                if eval < min_eval:
                    min_eval = eval
                    best_nod = fiu
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return min_eval, best_nod

    def rezolvare(self, algoritm, adancime):
        if algoritm == "MinMax":
            nod = self.minmax(self.nod_initial, adancime, True)
        elif algoritm == "AlphaBeta":
            _, nod = self.alphabeta(self.nod_initial, adancime, -float('inf'), float('inf'))
        else:
            print("Algoritm invalid")

        mutare = Game.MAX

        nodStart = nod.drumRadacina()[0]
        print(f"-- Starea initiala a jocului --\nScor: {nodStart.estimare()}")
        print(nodStart)

        for nod in nod.drumRadacina()[1:]:
            print(f"-- Mutarea lui {mutare} --\nScor: {nod.estimare()}")
            mutare = Game.jucator_opus(mutare)

            print(nod)

# Input si Rezolvare

In [5]:
stare_initiala = ['.'] * 24
stare_initiala[11] = 'x'
stare_initiala[12] = '0'
stare_initiala[13] = 'x'
stare_initiala[17] = '0'
stare_initiala[18] = 'x'
stare_initiala[19] = '0'


piese_de_pus = (6, 6)
graf = Graf(stare_initiala, piese_de_pus, Game.MAX)


#graf.rezolvare("AlphaBeta", 3)
graf.rezolvare("MinMax", 3)



-- Starea initiala a jocului --
Scor: -8
    .-----------.-----------.
    |           |           |
    |   .-------.-------.   |
    |   |       |       |   |
    |   |   .---.---.   |   |
    |   |   |       |   |   |
    .---.---x       0---x---.
    |   |   |       |   |   |
    |   |   .---.---0   |   |
    |   |       |       |   |
    |   x-------0-------.   |
    |           |           |
    .-----------.-----------.

Piese de pus: x=6, 0=6

-- Mutarea lui x --
Scor: 3
    .-----------.-----------.
    |           |           |
    |   .-------.-------.   |
    |   |       |       |   |
    |   |   .---.---x   |   |
    |   |   |       |   |   |
    .---.---x       0---x---.
    |   |   |       |   |   |
    |   |   .---.---0   |   |
    |   |       |       |   |
    |   x-------0-------.   |
    |           |           |
    .-----------.-----------.

Piese de pus: x=5, 0=6

-- Mutarea lui 0 --
Scor: -15
    .-----------.-----------.
    |           |           |
    |   .--

# Documentatie

# Justificare euristica
Euristica este reprezentata de scorul unei stari de joc. Am salvat in clasa Game toate configuratiile de pozitii ale morilor posibile in acest joc, iar pe baza acestor configurari, pot sa calculez euristica nodului (starii) astfel: 
- scor_secventa( ) calculeaza scorul unei mori date ca parametru: 10 ^ (nr simboluri in secventa - 1) daca jucatorul opus nu e in secventa, 0 altfel
- estimare( ) foloseste functia scor_secventa( ) pentru a calcula punctajul total al starii, adica suma tuturor configurarilor pozitiilor atat ale jucatorului MAX, cat si ale jucatorului MIN

Astfel, mutarile sunt determinate in functie de acest scor de estimare si de jucatorul curent care urmeaza sa mute

Algoritmii de MinMax si AlphaBeta determina starea jocului dupa adancimea data ca si parametru, fiecare jucator facand mutarea optima.